In [1]:
import xarray as xr
import numpy as np
import cftime
import nc_time_axis
import pandas as pd
import matplotlib.pyplot as plt 
from matplotlib import ticker, cm
from cftime import datetime 
import datetime as dt
import matplotlib.colors as colors
import math
import random
import matplotlib.cm as mcm
from matplotlib.ticker import MaxNLocator
#jet = mcm.get_cmap('jet')
jet = mcm.get_cmap('jet') if isinstance(mcm.get_cmap('jet'), str) else mcm.get_cmap('jet')
import netCDF4 as nc
import sys
import os
import psutil
import netCDF4 as nc
import cartopy.crs as ccrs # CRS stands for "Coordinate reference systems" for map projection
from cartopy.crs import PlateCarree
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from dateutil import tz
import pytz
import time
from time import process_time
from tqdm import tqdm
import dask.array as da
import dask.dataframe as dd
import dask
%matplotlib inline 
#import line_profiler
#%load_ext line_profiler

/tmp/ipykernel_691915/892673583.py:16: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  jet = mcm.get_cmap('jet') if isinstance(mcm.get_cmap('jet'), str) else mcm.get_cmap('jet')


In [2]:
### Transform raw satellite data into new data structure ###

filepath = 'Nc_Files/1_s4max.nc'
ds = xr.open_dataset(filepath)
ds

<xarray.Dataset>
Dimensions:   (nfile: 9266536)
Dimensions without coordinates: nfile
Data variables:
    Ut_time   (nfile) float64 ...
    altitude  (nfile) float64 ...
    DOY       (nfile) float64 ...
    lat       (nfile) float64 ...
    LT        (nfile) float64 ...
    lon       (nfile) float64 ...
    mlat      (nfile) float64 ...
    mlon      (nfile) float64 ...
    mlt       (nfile) float64 ...
    s4max     (nfile) float64 ...
    year      (nfile) float64 ...

In [3]:
# Create coordinates for latitude, longitude, altitude, and time
lat = ds['lat']
lon = ds['lon']
altitude = ds['altitude']


# Extract 'year' values from the DataArray
year_values = ds['year'].values.astype(int)

# Convert 'DOY' and 'Ut_time' to integers
doy = ds['DOY'].values.astype(int)
ut_time = ds['Ut_time'].values

# Create a list of datetime objects for each data point
time_list = []

# Initialize the progress bar
with tqdm(total=len(year_values)) as pbar:
    for y, d, h in zip(year_values, doy, ut_time):
        # Specify the correct format for 'year' and 'DOY'
        year_str = str(y)
        doy_str = f'{d:03d}'
        
        # Calculate hours and minutes from the decimal 'Ut_time'
        hours = int(h)
        minutes = int((h - hours) * 60)
        
        # Create the datetime object
        time_list.append(pd.to_datetime(f'{year_str}-{doy_str} {hours:02d}:{minutes:02d}', format='%Y-%j %H:%M'))
        
        # Update the progress bar
        pbar.update(1)

# Create a 'time' coordinate from the list
ds['time'] = xr.DataArray(time_list, dims=('nfile',))

100%|███████████████████████████████████████████████████████████████████████| 9266536/9266536 [17:49<00:00, 8665.93it/s]


In [12]:
 
# Create a 'season' coordinate based on 'DOY' with specified dates, and specify its data type
ds['season'] = xr.where(
    (ds['DOY'] >= 60) & (ds['DOY'] <= 151), 'Spring',  # March 1 to May 31
    xr.where(
        (ds['DOY'] >= 152) & (ds['DOY'] <= 243), 'Summer',  # June 1 to August 31
        xr.where(
            (ds['DOY'] >= 244) & (ds['DOY'] <= 334), 'Autumn',  # September 1 to November 30
            'Winter'  # December 1 to February 28/29
        )
    ),
)



# Create a 'month' coordinate based on 'DOY'
ds['month'] = xr.where((ds['DOY'] >= 1) & (ds['DOY'] <= 31), 'January',
                       xr.where((ds['DOY'] >= 32) & (ds['DOY'] <= 59), 'February',
                                xr.where((ds['DOY'] >= 60) & (ds['DOY'] <= 90), 'March',
                                         xr.where((ds['DOY'] >= 91) & (ds['DOY'] <= 120), 'April',
                                                  xr.where((ds['DOY'] >= 121) & (ds['DOY'] <= 151), 'May',
                                                           xr.where((ds['DOY'] >= 152) & (ds['DOY'] <= 181), 'June',
                                                                    xr.where((ds['DOY'] >= 182) & (ds['DOY'] <= 212), 'July',
                                                                             xr.where((ds['DOY'] >= 213) & (ds['DOY'] <= 243), 'August',
                                                                                      xr.where((ds['DOY'] >= 244) & (ds['DOY'] <= 273), 'September',
                                                                                               xr.where((ds['DOY'] >= 274) & (ds['DOY'] <= 304), 'October',
                                                                                                        xr.where((ds['DOY'] >= 305) & (ds['DOY'] <= 334), 'November',
                                                                                                                 'December')))))))))))

In [13]:
# Create the  coordinates as variables
ds['lat_var'] = xr.DataArray(ds['lat'].values, dims='nfile')
ds['lon_var'] = xr.DataArray(ds['lon'].values, dims='nfile')
ds['altitude_var'] = xr.DataArray(ds['altitude'].values, dims='nfile')
ds['time_var'] = xr.DataArray(ds['time'].values, dims='nfile')
ds['season_var'] = xr.DataArray(ds['season'].values, dims='nfile')
ds['month_var'] = xr.DataArray(ds['month'].values, dims='nfile')


ds = ds.set_coords(['lon', 'lat', 'altitude', 'time', 'season', 'month'])

# Specify the path for the new netCDF file
new_filepath = 'Nc_Files/mod_s4max.nc'


# Check if the file already exists
if os.path.exists(new_filepath):
    # If it exists, delete it
    os.remove(new_filepath)


# Save the modified dataset to the new file
ds.to_netcdf(new_filepath)

# Close the dataset to release resources
ds.close()

In [18]:
new_filepath = 'Nc_Files/mod_s4max.nc'

ds2 = xr.open_dataset(new_filepath)

ds2


<xarray.Dataset>
Dimensions:       (nfile: 9266536)
Coordinates:
    altitude      (nfile) float64 ...
    lat           (nfile) float64 ...
    lon           (nfile) float64 ...
    time          (nfile) datetime64[ns] ...
    season        (nfile) object ...
    month         (nfile) object ...
Dimensions without coordinates: nfile
Data variables: (12/14)
    Ut_time       (nfile) float64 ...
    DOY           (nfile) float64 ...
    LT            (nfile) float64 ...
    mlat          (nfile) float64 ...
    mlon          (nfile) float64 ...
    mlt           (nfile) float64 ...
    ...            ...
    month_var     (nfile) object ...
    season_var    (nfile) object ...
    lat_var       (nfile) float64 ...
    lon_var       (nfile) float64 ...
    altitude_var  (nfile) float64 ...
    time_var      (nfile) datetime64[ns] ...

In [19]:
ds2_min_altitude = ds2['altitude'].min().values
ds2_max_altitude = ds2['altitude'].max().values

print("Minimum Altitude:", ds2_min_altitude)
print("Maximum Altitude:", ds2_max_altitude)

Minimum Altitude: 40.0
Maximum Altitude: 876.3590087890625


In [20]:
# Get available CPU cores
num_cores = os.cpu_count()

# Get available RAM in GB
ram_gb = round(psutil.virtual_memory().total / (1024 ** 3), 2)

num_cores, ram_gb

(80, 125.26)

In [21]:
# Define altitude_min and altitude_max
altitude_min = 80.0  # Minimum altitude in kilometers
altitude_max = 130.0  # Maximum altitude in kilometers

# Create a Dask DataFrame or Dask Array from your xarray dataset
# Specify the chunks parameter for efficient parallel processing
# Example for a Dask DataFrame:
ddf = dd.from_pandas(ds2.to_dataframe(), npartitions=80)  # Use 80 partitions to match CPU cores

# Define the slicing conditions as Dask objects
condition_min = ddf['altitude'] >= altitude_min
condition_max = ddf['altitude'] <= altitude_max

# Combine conditions if needed (e.g., for both altitude_min and altitude_max)
combined_condition = condition_min & condition_max

# Use Dask to perform the slicing in parallel
sliced_ddf = ddf[combined_condition]

# Compute the result if needed
result = sliced_ddf.compute()

In [22]:
result_min_altitude = result['altitude'].min()
result_max_altitude = result['altitude'].max()

print("Minimum Altitude:", result_min_altitude)
print("Maximum Altitude:", result_max_altitude)

Minimum Altitude: 80.0
Maximum Altitude: 130.0


In [23]:
result

,Ut_time,altitude,DOY,lat,LT,lon,mlat,mlon,mlt,s4max,year,time,season,month,month_var,season_var,lat_var,lon_var,altitude_var,time_var
nfile,,,,,,,,,,,,,,,,,,,,
0,21.933333,110.728996,362.0,-40.039001,11.540000,-163.597000,-41.650055,-80.584921,11.635215,1.366,2006.0,2006-12-28 21:56:00,Winter,December,December,Winter,-40.039001,-163.597000,110.728996,2006-12-28 21:56:00
2,10.166667,114.486000,362.0,-45.904999,11.815000,16.735001,-48.902820,74.528628,10.209452,0.376,2006.0,2006-12-28 10:09:00,Winter,December,December,Winter,-45.904999,16.735001,114.486000,2006-12-28 10:09:00
3,22.150000,119.097000,362.0,-55.358002,16.188000,-96.500000,-42.797303,-13.678671,16.312299,1.464,2006.0,2006-12-28 22:08:00,Winter,December,December,Winter,-55.358002,-96.500000,119.097000,2006-12-28 22:08:00
4,20.166667,100.129997,362.0,-20.113001,11.761000,-133.882004,-16.791011,-58.698708,11.327630,0.618,2006.0,2006-12-28 20:10:00,Winter,December,December,Winter,-20.113001,-133.882004,100.129997,2006-12-28 20:10:00
6,12.433333,90.218002,362.0,-10.008000,22.073000,137.184998,-18.913609,-150.554754,21.470560,0.246,2006.0,2006-12-28 12:26:00,Winter,December,December,Winter,-10.008000,137.184998,90.218002,2006-12-28 12:26:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9266526,2.233333,98.013000,30.0,-43.720001,12.539000,154.524002,-53.556169,-124.370731,13.016161,0.349,2014.0,2014-01-30 02:14:00,Winter,January,January,Winter,-43.720001,154.524002,98.013000,2014-01-30 02:14:00
9266527,12.833333,89.532997,30.0,-7.578000,20.587999,116.209000,-16.776261,-172.112928,20.433348,0.072,2014.0,2014-01-30 12:50:00,Winter,January,January,Winter,-7.578000,116.209000,89.532997,2014-01-30 12:50:00
9266531,17.266667,103.887001,30.0,-67.353996,14.815000,-36.931000,-54.158198,24.699415,13.987504,0.952,2014.0,2014-01-30 17:15:00,Winter,January,January,Winter,-67.353996,-36.931000,103.887001,2014-01-30 17:15:00


In [53]:
#Just a check to look at the data

identifier = 'lat'  #can also use lat, lon, altitude, season

duplicates = result.duplicated(subset=[identifier])
num_duplicates = duplicates.sum()
total_data_points = len(result)
unique_values = total_data_points - num_duplicates

print(f"Total number of data points: {total_data_points}")
print(f"Number of duplicates: {num_duplicates}")
print(f"Number of unique values: {unique_values}")
print(' ')
unique_id = result[identifier].unique()
print("Unique "+identifier+"s:", unique_id)


Total number of data points: 6449380
Number of duplicates: 6270997
Number of unique values: 178383
 
Unique lats: [-40.03900146 -45.90499878 -55.35800171 ... -89.11599731 -86.61299896
 -88.24500275]


In [27]:
ds3 = xr.Dataset.from_dataframe(result)

ds3 = ds3.set_coords(['lon', 'lat', 'altitude', 'season', 'time', 'month'])


result_filepath = 'Nc_Files/mod_s4max_80-130km.nc'

# Check if the file already exists
if os.path.exists(result_filepath):
    # If it exists, delete it
    os.remove(result_filepath)

# Save the Dataset to a NetCDF file
ds3.to_netcdf(result_filepath)

In [2]:
result_filepath = 'Nc_Files/mod_s4max_80-130km.nc'

dss = xr.open_dataset(result_filepath)

dss

<xarray.Dataset>
Dimensions:       (nfile: 6449380)
Coordinates:
  * nfile         (nfile) int64 0 2 3 4 6 ... 9266527 9266531 9266534 9266535
    altitude      (nfile) float64 ...
    lat           (nfile) float64 ...
    lon           (nfile) float64 ...
    time          (nfile) datetime64[ns] ...
    season        (nfile) object ...
    month         (nfile) object ...
Data variables: (12/14)
    Ut_time       (nfile) float64 ...
    DOY           (nfile) float64 ...
    LT            (nfile) float64 ...
    mlat          (nfile) float64 ...
    mlon          (nfile) float64 ...
    mlt           (nfile) float64 ...
    ...            ...
    month_var     (nfile) object ...
    season_var    (nfile) object ...
    lat_var       (nfile) float64 ...
    lon_var       (nfile) float64 ...
    altitude_var  (nfile) float64 ...
    time_var      (nfile) datetime64[ns] ...

In [3]:
# Get the unique values for dimensions
unique_lat = np.unique(dss['lat'])
unique_lon = np.unique(dss['lon'])
unique_altitude = np.unique(dss['altitude'])
unique_time = np.unique(dss['time'])
unique_month = np.unique(dss['month'])
unique_season = np.unique(dss['season'])


# print("Unique Lat Values:", unique_lat)
# print("Number of Unique Lat Values:", len(unique_lat))
# print("-------------------------------------------------------")

# print("Unique Lon Values:", unique_lon)
# print("Number of Unique Lon Values:", len(unique_lon))
# print("-------------------------------------------------------")

# print("Unique Altitude Values:", unique_altitude)
# print("Number of Unique Altitude Values:", len(unique_altitude))
# print("-------------------------------------------------------")

# print("Unique Time Values:", unique_time)
# print("Number of Unique Time Values:", len(unique_time))
# print("-------------------------------------------------------")

# print("Unique Month Values:", unique_month)
# print("Number of Unique Month Values:", len(unique_month))
# print("-------------------------------------------------------")

# print("Unique Season Values:", unique_season)
# print("Number of Unique Season Values:", len(unique_season))

In [4]:
# # Create an empty s4max variable

# s4max_dims = ('lat', 'lon', 'altitude', 'time')#, 'month', 'season')
# s4max_shape = (
#     len(unique_lat),
#     len(unique_lon),
#     len(unique_altitude),
#     len(unique_time),
#     #len(unique_month),
#     #len(unique_season),
# )
# new_s4max = xr.DataArray(np.zeros(s4max_shape), dims=s4max_dims)

In [5]:
# Calculate the shape for your new s4max variable
s4max_shape = (
    len(unique_lat),
    len(unique_lon),
    len(unique_altitude),
    len(unique_time),
    len(unique_month),
    len(unique_season)
)

# Create a Dask array with zeros (use da.zeros instead of np.zeros)
new_s4max_dask = da.zeros(s4max_shape, chunks='auto')

In [6]:
# Convert the Dask array to an xarray DataArray
new_s4max = xr.DataArray(new_s4max_dask, dims=('lat', 'lon', 'altitude', 'time', 'month', 'season'))

In [ ]:
# Populate the new_s4max variable
for i, row in dss.to_dataframe().iterrows():
    lat = row['lat']
    lon = row['lon']
    altitude = row['altitude']
    time = row['time']
    month = row['month']
    season = row['season']
    s4max = row['s4max']

    lat_idx = np.where(unique_lat == lat)[0]
    lon_idx = np.where(unique_lon == lon)[0]
    altitude_idx = np.where(unique_altitude == altitude)[0]
    time_idx = np.where(unique_time == time)[0]
    month_idx = np.where(unique_month == month)[0]
    season_idx = np.where(unique_season == season)[0]

    new_s4max[lat_idx, lon_idx, altitude_idx, time_idx, month_idx, season_idx] = s4max

# Now you have a new_s4max variable with dimensions based on unique coordinate values.


In [ ]:
# Compute the Dask array
new_s4max = new_s4max.compute()

In [37]:
stacked_ds = dss.stack(alt_dim=('altitude',))
stacked_ds = stacked_ds.stack(lat_dim=('lat',))
stacked_ds = stacked_ds.stack(lon_dim=('lon',))
stacked_ds = stacked_ds.stack(time_dim=('time',))
stacked_ds = stacked_ds.stack(season_dim=('season',))
stacked_ds = stacked_ds.stack(month_dim=('month',))


In [38]:
stacked_ds

<xarray.Dataset>
Dimensions:       (nfile: 6449380, alt_dim: 6449380, lat_dim: 6449380,
                   lon_dim: 6449380, time_dim: 6449380, season_dim: 6449380,
                   month_dim: 6449380)
Coordinates: (12/13)
  * nfile         (nfile) int64 0 2 3 4 6 ... 9266527 9266531 9266534 9266535
  * alt_dim       (alt_dim) object MultiIndex
  * altitude      (alt_dim) float64 110.7 114.5 119.1 100.1 ... 103.9 84.5 95.78
  * lat_dim       (lat_dim) object MultiIndex
  * lat           (lat_dim) float64 -40.04 -45.9 -55.36 ... -67.35 45.36 40.25
  * lon_dim       (lon_dim) object MultiIndex
    ...            ...
  * time_dim      (time_dim) object MultiIndex
  * time          (time_dim) datetime64[ns] 2006-12-28T21:56:00 ... 2014-01-3...
  * season_dim    (season_dim) object MultiIndex
  * season        (season_dim) object 'Winter' 'Winter' ... 'Winter' 'Winter'
  * month_dim     (month_dim) object MultiIndex
  * month         (month_dim) object 'December' 'December' ... 'January'
Data variables: (12/14)
    Ut_time       (nfile) float64 ...
    DOY           (nfile) float64 ...
    LT            (nfile) float64 ...
    mlat          (nfile) float64 ...
    mlon          (nfile) float64 ...
    mlt           (nfile) float64 ...
    ...            ...
    month_var     (nfile) object ...
    season_var    (nfile) object ...
    lat_var       (nfile) float64 ...
    lon_var       (nfile) float64 ...
    altitude_var  (nfile) float64 ...
    time_var      (nfile) datetime64[ns] ...

In [38]:
stacked_ds['s4max'] = stacked_ds['s4max'].unstack('lat_dim')
stacked_ds['s4max'] = stacked_ds['s4max'].unstack('lat_dim')
stacked_ds['s4max'] = stacked_ds['s4max'].unstack('lon_dim')
stacked_ds['s4max'] = stacked_ds['s4max'].unstack('time_dim')
stacked_ds['s4max'] = stacked_ds['s4max'].unstack('month_dim')
stacked_ds['s4max'] = stacked_ds['s4max'].unstack('season_dim')


ValueError: Dimensions ('lat_dim',) not found in data dimensions ('nfile',)

In [39]:
stacked_ds

<xarray.Dataset>
Dimensions:     (nfile: 6449380, lat_dim: 6449380, lon_dim: 6449380,
                 time_dim: 6449380, month_dim: 6449380, season_dim: 6449380)
Coordinates:
  * nfile       (nfile) int64 0 2 3 4 6 ... 9266527 9266531 9266534 9266535
    altitude    (nfile) float64 ...
  * lat_dim     (lat_dim) object MultiIndex
  * lat         (lat_dim) float64 -40.04 -45.9 -55.36 ... -67.35 45.36 40.25
  * lon_dim     (lon_dim) object MultiIndex
  * lon         (lon_dim) float64 -163.6 16.74 -96.5 ... -36.93 -163.4 150.5
  * time_dim    (time_dim) object MultiIndex
  * time        (time_dim) datetime64[ns] 2006-12-28T21:56:00 ... 2014-01-30T...
  * month_dim   (month_dim) object MultiIndex
  * month       (month_dim) object 'December' 'December' ... 'January' 'January'
  * season_dim  (season_dim) object MultiIndex
  * season      (season_dim) object 'Winter' 'Winter' ... 'Winter' 'Winter'
Data variables:
    Ut_time     (nfile) float64 ...
    DOY         (nfile) float64 ...
    LT          (nfile) float64 ...
    mlat        (nfile) float64 ...
    mlon        (nfile) float64 ...
    mlt         (nfile) float64 ...
    s4max       (nfile) float64 ...
    year        (nfile) float64 ...

In [34]:
ds = xr.Dataset(data_vars={'s4max': (('nfile', 'lat', 'lon', 'altitude', 'time'), result['s4max'].values)},
                coords={'lat': result['lat'].values,
                        'lon': result['lon'].values,
                        'altitude': result['altitude'].values,
                        'time': result['time'].values})

# You can add more variables in a similar manner
# e.g., ds['variable_name'] = (('nfile', 'lat', 'lon', 'altitude', 'time'), result['variable_name'].values)

# Now, your dataset is multidimensional

ds

ValueError: Variable 's4max': Could not convert tuple of form (dims, data[, attrs, encoding]): (('nfile', 'lat', 'lon', 'altitude', 'time'), array([1.36600006, 0.37599999, 1.46399999, ..., 0.95200002, 0.075     ,
       0.07      ])) to Variable.

In [5]:
xarray_data = stacked_ds.reset_index('lat_dim')  # Replace 'lat_dim' with the desired dimension
xarray_data

<xarray.Dataset>
Dimensions:     (nfile: 6449380, lat_dim: 6449380, lon_dim: 6449380,
                 time_dim: 6449380, month_dim: 6449380, season_dim: 6449380)
Coordinates:
  * nfile       (nfile) int64 0 2 3 4 6 ... 9266527 9266531 9266534 9266535
    altitude    (nfile) float64 ...
    lat         (lat_dim) float64 -40.04 -45.9 -55.36 ... -67.35 45.36 40.25
  * lon_dim     (lon_dim) object MultiIndex
  * lon         (lon_dim) float64 -163.6 16.74 -96.5 ... -36.93 -163.4 150.5
  * time_dim    (time_dim) object MultiIndex
  * time        (time_dim) datetime64[ns] 2006-12-28T21:56:00 ... 2014-01-30T...
  * month_dim   (month_dim) object MultiIndex
  * month       (month_dim) object 'December' 'December' ... 'January' 'January'
  * season_dim  (season_dim) object MultiIndex
  * season      (season_dim) object 'Winter' 'Winter' ... 'Winter' 'Winter'
Dimensions without coordinates: lat_dim
Data variables:
    Ut_time     (nfile) float64 ...
    DOY         (nfile) float64 ...
    LT          (nfile) float64 ...
    mlat        (nfile) float64 ...
    mlon        (nfile) float64 ...
    mlt         (nfile) float64 ...
    s4max       (nfile) float64 ...
    year        (nfile) float64 ...

In [ ]:
############## OLD CODE ###################

In [99]:
# Create a DataFrame
df = pd.DataFrame({'year': year, 'DOY': DOY, 'Ut_time': Ut_time})

# Group the data by 'year' and 'DOY' and count measurements for each day
daily_counts = df.groupby(['year', 'DOY']).size().reset_index(name='measurement_count')

# Print the daily counts
print(daily_counts)

        year    DOY  measurement_count
0     2006.0  362.0                557
1     2006.0  363.0               1073
2     2006.0  364.0                855
3     2006.0  365.0                973
4     2007.0    1.0                918
...      ...    ...                ...
2556  2014.0   26.0               2220
2557  2014.0   27.0               2332
2558  2014.0   28.0               2131
2559  2014.0   29.0               1245
2560  2014.0   30.0               1882

[2561 rows x 3 columns]


In [4]:
# Create a DataFrame
df = pd.DataFrame({'year': year, 'DOY': DOY, 'Ut_time': Ut_time, 's4max': s4max})

# Group the data by 'year' and 'DOY' and calculate the daily average of 's4max'
daily_average_s4max = df.groupby(['year', 'DOY'])['s4max'].mean().reset_index()

# Print the daily average s4max values
print(daily_average_s4max)

        year    DOY     s4max
0     2006.0  362.0  0.398154
1     2006.0  363.0  0.472669
2     2006.0  364.0  0.462701
3     2006.0  365.0  0.477520
4     2007.0    1.0  0.496682
...      ...    ...       ...
2556  2014.0   26.0  0.332223
2557  2014.0   27.0  0.353242
2558  2014.0   28.0  0.353679
2559  2014.0   29.0  0.376080
2560  2014.0   30.0  0.352309

[2561 rows x 3 columns]


In [5]:
# Extract month from 'DOY' and store it in a new column 'month'
df['month'] = pd.to_datetime(df['DOY'], format='%j').dt.month

# Group the data by 'year' and 'month' and calculate the monthly average of 's4max'
monthly_average_s4max = df.groupby(['year', 'month'])['s4max'].mean().reset_index()

# Print the monthly average s4max values
print(monthly_average_s4max)

      year  month     s4max
0   2006.0     12  0.459567
1   2007.0      1  0.414799
2   2007.0      2  0.367987
3   2007.0      3  0.309287
4   2007.0      4  0.348409
..     ...    ...       ...
81  2013.0      9  0.265695
82  2013.0     10  0.292686
83  2013.0     11  0.348491
84  2013.0     12  0.369266
85  2014.0      1  0.341636

[86 rows x 3 columns]


In [6]:
# Extract month from 'DOY' and store it in a new column 'month'
df['month'] = pd.to_datetime(df['DOY'], format='%j').dt.month

# Create a new column 'season' to represent three-month periods
df['season'] = df['month'].apply(lambda x: 
    'Mar-May' if 3 <= x <= 5 else
    'Jun-Aug' if 6 <= x <= 8 else
    'Sep-Nov' if 9 <= x <= 11 else
    'Dec-Feb'
)

# Group the data by 'year' and 'season' and calculate the three-monthly average of 's4max'
three_monthly_average_s4max = df.groupby(['year', 'season'])['s4max'].mean().reset_index()

# Print the three-monthly average s4max values
print(three_monthly_average_s4max)

      year   season     s4max
0   2006.0  Dec-Feb  0.459567
1   2007.0  Dec-Feb  0.401703
2   2007.0  Jun-Aug  0.436065
3   2007.0  Mar-May  0.367342
4   2007.0  Sep-Nov  0.325800
5   2008.0  Dec-Feb  0.444378
6   2008.0  Jun-Aug  0.397703
7   2008.0  Mar-May  0.294320
8   2008.0  Sep-Nov  0.279008
9   2009.0  Dec-Feb  0.429219
10  2009.0  Jun-Aug  0.417608
11  2009.0  Mar-May  0.320908
12  2009.0  Sep-Nov  0.309640
13  2010.0  Dec-Feb  0.372818
14  2010.0  Jun-Aug  0.398084
15  2010.0  Mar-May  0.332921
16  2010.0  Sep-Nov  0.267643
17  2011.0  Dec-Feb  0.390770
18  2011.0  Jun-Aug  0.411044
19  2011.0  Mar-May  0.308562
20  2011.0  Sep-Nov  0.331371
21  2012.0  Dec-Feb  0.399845
22  2012.0  Jun-Aug  0.394944
23  2012.0  Mar-May  0.319996
24  2012.0  Sep-Nov  0.340638
25  2013.0  Dec-Feb  0.338472
26  2013.0  Jun-Aug  0.371956
27  2013.0  Mar-May  0.312531
28  2013.0  Sep-Nov  0.303845
29  2014.0  Dec-Feb  0.341636


In [133]:
# Calculate the datetime based on 'year,' 'DOY,' and 'Ut_time' in hours
hours_per_day = 24

def create_datetime(year, DOY, Ut_time):
    date_str = []
    for y, d, t in zip(year, DOY, Ut_time):
        date_str.append(f"{int(y)}-{int(d):03d} {t:02.3f}:00:00")
    return date_str

ds['datetime'] = xr.apply_ufunc(create_datetime, ds['year'], ds['DOY'], ds['Ut_time'],
                                dask='parallelized', output_dtypes=['object'])

# Convert the 'datetime' column to pandas datetime
ds['datetime'] = pd.to_datetime(ds['datetime'], format='%Y-%j %H.%f:%M:%S')

# Calculate the daily average of 's4max'
daily_average_s4max = ds['s4max'].resample(datetime='D').mean(dim='datetime')

# Optionally, you can convert the result to a pandas DataFrame
daily_average_s4max_df = daily_average_s4max.to_dataframe()

print(daily_average_s4max_df)

KeyError: 'datetime'